In [1]:
import pandas as pd
import numpy as np

## **Objetivo**

Realizar la ETL de la base maestra.

**Pasos**

- Merge la BDM de SAIGC para obtener la información del ICFES
- Corregir irregularidades con los créditos consumidos:
- 1.  Los créditos que están vacíos en términos de consumo (suma de creditos perdidos y aprobados) imputarlos con el consumo teórico (creditos programa / periodos) * número de matriculas. Asumir que el 98% era aprobación y el 2% de pérdida.
- 2. Cuando el número de créditos consumidos no es igual a la suma de créditos pérdidos y aprobados, entonces ajustar el número de créditos consumidos con el de aprobados.  

In [2]:
df_maestra_posmedia = pd.read_excel("../data/BD_Maestra_2025_10_15.xlsx", "BDF")
df_maestra_SAIGC = pd.read_excel("../data/20250930_SAIGC_MAESTRA_JE_BENEFICIARIOS_V9.xlsx") 

In [82]:
df_maestra_posmedia_v2 = df_maestra_posmedia[df_maestra_posmedia["CUPO"] == "ACTIVO"]

df_maestra_SAIGC_unicos = (
    df_maestra_SAIGC
    .drop_duplicates(subset="ID_PERSONA", keep="first")
    [["ID_PERSONA", "SABER11_PUNTAJE_GLOBAL"]])

df_maestra_posmedia_v3 = df_maestra_posmedia_v2.merge(
    df_maestra_SAIGC_unicos,
    on="ID_PERSONA",
    how="left")

df_maestra_posmedia_v3["SABER11_IMPUTADO"] = df_maestra_posmedia_v3["SABER11_PUNTAJE_GLOBAL"].isna()

df_maestra_posmedia_v3["SABER11_PUNTAJE_GLOBAL"] = (
    df_maestra_posmedia_v3
    .groupby(["CONVOCATORIA", "NIVEL_FORMACION"])["SABER11_PUNTAJE_GLOBAL"]
    .transform(lambda x: x.fillna(x.mean())))

df_maestra_posmedia_v3["SABER11_IMPUTADO"] = np.where(
    df_maestra_posmedia_v3["SABER11_IMPUTADO"], "IMPUTADO", "ORIGINAL")

In [83]:
# Inicializa validador
df_maestra_posmedia_v3["validador"] = np.where(
    df_maestra_posmedia_v3["ACU_CRED_CONSUMIDOS"] ==
    df_maestra_posmedia_v3["ACU_CRED_APROBADOS"] + df_maestra_posmedia_v3["ACU_CRED_PERDIDOS"],
    "OK",
    "AJUSTADO"
)

# Ajuste 1: inconsistencia general
df_maestra_posmedia_v3.loc[df_maestra_posmedia_v3["validador"] == "AJUSTADO", "ACU_CRED_CONSUMIDOS"] = (
    df_maestra_posmedia_v3["ACU_CRED_APROBADOS"] + df_maestra_posmedia_v3["ACU_CRED_PERDIDOS"]
)
df_maestra_posmedia_v3.loc[df_maestra_posmedia_v3["validador"] == "AJUSTADO", "validador"] = "AJUSTADO_GENERAL"

# Ajuste 2: consumidos=0/Na y perdidos>0
cond1 = (
    ((df_maestra_posmedia_v3["ACU_CRED_CONSUMIDOS"].isna()) | (df_maestra_posmedia_v3["ACU_CRED_CONSUMIDOS"] == 0)) &
    ((df_maestra_posmedia_v3["ACU_CRED_APROBADOS"].isna()) | (df_maestra_posmedia_v3["ACU_CRED_APROBADOS"] == 0)) &
    (df_maestra_posmedia_v3["ACU_CRED_PERDIDOS"] > 0)
)
df_maestra_posmedia_v3.loc[cond1, "ACU_CRED_CONSUMIDOS"] = (
    df_maestra_posmedia_v3["ACU_CRED_APROBADOS"].fillna(0) +
    df_maestra_posmedia_v3["ACU_CRED_PERDIDOS"].fillna(0)
)
df_maestra_posmedia_v3.loc[cond1, "validador"] = "AJUSTADO_PERDIDOS"

# Ajuste 3: todos 0/Na → imputación teórica
cond2 = (
    ((df_maestra_posmedia_v3["ACU_CRED_CONSUMIDOS"].isna()) | (df_maestra_posmedia_v3["ACU_CRED_CONSUMIDOS"] == 0)) &
    ((df_maestra_posmedia_v3["ACU_CRED_APROBADOS"].isna()) | (df_maestra_posmedia_v3["ACU_CRED_APROBADOS"] == 0)) &
    ((df_maestra_posmedia_v3["ACU_CRED_PERDIDOS"].isna()) | (df_maestra_posmedia_v3["ACU_CRED_PERDIDOS"] == 0))
)

base = (
    (df_maestra_posmedia_v3["CREDITOS_PROGRAMA"] / df_maestra_posmedia_v3["PERIODOS_BD_SNIES"])
    * df_maestra_posmedia_v3["CUENTA_MATRICULADO"]
)
df_maestra_posmedia_v3.loc[cond2, "ACU_CRED_APROBADOS"] = (base * 0.95).fillna(0).astype(int)
df_maestra_posmedia_v3.loc[cond2, "ACU_CRED_PERDIDOS"] = (base * 0.05).fillna(0).astype(int)
df_maestra_posmedia_v3.loc[cond2, "ACU_CRED_CONSUMIDOS"] = (
    df_maestra_posmedia_v3["ACU_CRED_APROBADOS"] + df_maestra_posmedia_v3["ACU_CRED_PERDIDOS"]
)
df_maestra_posmedia_v3.loc[cond2, "validador"] = "AJUSTADO_IMPUTADO"


In [84]:
# Crear columna de control para registrar los reemplazos
df_maestra_posmedia_v3["REEMPLAZO_ESTADO"] = ""

# --- Reemplazo para ESTADO_20242_CIERRE ---
cond_20242 = df_maestra_posmedia_v3["ESTADO_20242_CIERRE"] == "EN VERIFICACION"
df_maestra_posmedia_v3.loc[cond_20242, "REEMPLAZO_ESTADO"] = "ESTADO_20242_CIERRE → ULTIMO_ESTADO"
df_maestra_posmedia_v3.loc[cond_20242, "ESTADO_20242_CIERRE"] = df_maestra_posmedia_v3.loc[cond_20242, "ULTIMO_ESTADO"]

# --- Reemplazo para ESTADO_20251_CIERRE ---
cond_20251 = df_maestra_posmedia_v3["ESTADO_20251_CIERRE"] == "EN VERIFICACION"
df_maestra_posmedia_v3.loc[cond_20251, "REEMPLAZO_ESTADO"] = (
    df_maestra_posmedia_v3["REEMPLAZO_ESTADO"].astype(str).replace("", np.nan) + 
    " | ESTADO_20251_CIERRE → ULTIMO_ESTADO"
).fillna("ESTADO_20251_CIERRE → ULTIMO_ESTADO")
df_maestra_posmedia_v3.loc[cond_20251, "ESTADO_20251_CIERRE"] = df_maestra_posmedia_v3.loc[cond_20251, "ULTIMO_ESTADO"]

**Crear las columnas "ESTADO_20253_INICIO" y "ESTADO_20251_CIERRE"**

In [ ]:
df_maestra_posmedia_v3.to_excel("../output/BASE_MAESTRA_AJUSTADA_BMD15102025.xlsx")